# Imports


In [1]:
import os
import cv2
import glob

import numpy as np

from PIL import Image
from scipy.io import loadmat

# Create .npy files


## Input and output paths


In [2]:
# Input paths
frames_path = "mall_dataset/frames/"
ground_truth_path = 'mall_dataset/ground_truth.mat'
perspective_roi_path = 'mall_dataset/perspective_roi.mat'

In [3]:
# Output paths
npy_files_path = 'npy_files'
os.makedirs(npy_files_path, exist_ok=True)

images_path = os.path.join(npy_files_path, 'images.npy')
images_gt_path = os.path.join(npy_files_path, 'images_gt.npy')
images_density_path = os.path.join(npy_files_path, 'images_density.npy')
labels_path = os.path.join(npy_files_path, 'labels.npy')

## Create images.npy


In [4]:
images = []
image_paths = sorted(glob.glob(frames_path + '*.jpg'))
for img_path in image_paths:
    img = Image.open(img_path)
    img = np.array(img)
    images.append(img)

images = np.array(images)

np.save(images_path, images)

## Create images_gt.npy from ground_truth.mat file.


In [5]:
ground_truth_data = loadmat(ground_truth_path)

frame_annotations = ground_truth_data['frame']

images_gt = []
for frame_data in frame_annotations[0]:
    if 'loc' in frame_data.dtype.names:
        frame_gt = frame_data['loc'][0, 0]
        images_gt.append(frame_gt)

images_gt = np.array(images_gt, dtype=object)

np.save(images_gt_path, images_gt)

## Create images_density.npy from perspective_roi.mat file.


In [6]:
perspective_roi_data = loadmat(perspective_roi_path)

roi = perspective_roi_data['roi'][0, 0][0]
pMapN = perspective_roi_data['pMapN']

roi = np.array(roi, dtype=np.float64)
if np.max(roi) > 1:
    roi /= np.max(roi)

pMapN_masked = pMapN * roi
pMapN_masked /= np.max(pMapN_masked)

In [7]:
images = np.load(images_path)
images_gt = np.load(images_gt_path, allow_pickle=True)

images_density = []
for frame_id, gt_points in enumerate(images_gt):
    density_map = np.zeros((images[frame_id].shape[0], images[frame_id].shape[1]), dtype=np.float32)

    for pt in gt_points:
        x, y = int(pt[0]), int(pt[1])
        if 0 <= x < density_map.shape[1] and 0 <= y < density_map.shape[0]:
            temp_map = np.zeros_like(density_map, dtype=np.float32)
            temp_map[y, x] = 1
            density_map += cv2.GaussianBlur(temp_map, (15, 15), sigmaX=4)

    images_density.append(density_map)

images_density = np.array(images_density)
np.save(images_density_path, images_density)

## Create labels.npy from images_gt.npy


In [8]:
images_gt = np.load(images_gt_path, allow_pickle=True)

labels = np.array([len(frame) for frame in images_gt])

np.save(labels_path, labels)